<a href="https://www.kaggle.com/code/parhamgousheh/ovarian-cancer?scriptVersionId=145925069" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
import cv2
import random
import os
import glob
import torch
import torch.nn as nn

from skimage import io
from PIL import Image
from sklearn.model_selection import train_test_split


In [2]:
pd.read_csv('/kaggle/input/UBC-OCEAN/train.csv').shape

(538, 5)

In [3]:
pd.read_csv('/kaggle/input/UBC-OCEAN/train.csv').head()

,image_id,label,image_width,image_height,is_tma
0,4,HGSC,23785,20008,False
1,66,LGSC,48871,48195,False
2,91,HGSC,3388,3388,True
3,281,LGSC,42309,15545,False
4,286,EC,37204,30020,False


In [4]:
# image_data = []
# image_label = []

# no_tma_path = "/kaggle/working/resized_train_no_tma"
# tma_path = "/kaggle/working/resized_train_tma"

# tumbnail_path = "/kaggle/input/UBC-OCEAN/train_thumbnails"
# train_path = "/kaggle/input/UBC-OCEAN/train_images"

# os.makedirs(tma_path, exist_ok=True)
# os.makedirs(no_tma_path, exist_ok=True)


# for img , label in zip(train_df_no_tma['image_id_path'],train_df_no_tma['label']):
#     image = Image.open(os.path.join(tumbnail_path, img))
#     image = image.resize((224,224))
#     image = image.convert("RGB")
#     save_path = f"{no_tma_path}/{img}"
#     image.save(save_path)

# for img , label in zip(train_df_tma['image_id_path'],train_df_tma['label']):
#     image = Image.open(os.path.join(train_path, img))
#     image = image.resize((224,224))
#     image = image.convert("RGB")
#     save_path = f"{tma_path}/{img}"
#     image.save(save_path)

In [5]:
df = pd.read_csv("/kaggle/input/UBC-OCEAN/train.csv")
print("Unique Labels and Counts:")
print(df['label'].value_counts())

Unique Labels and Counts:
label
HGSC    222
EC      124
CC       99
LGSC     47
MC       46
Name: count, dtype: int64


In [6]:
# plt.figure(figsize=(16,24))
# path = "/kaggle/input/UBC-OCEAN/train_thumbnails"
# j=1
# for img, lb in zip(train_df_no_tma['image_id_path'][:24],train_df_no_tma['label'][:24]):
#     plt.subplot(6,4,j)
#     path = os.path.join("/kaggle/input/UBC-OCEAN/train_thumbnails/",img)
#     image = plt.imread(path)
#     image = plt.imshow(image)
#     plt.title(f"Label:{lb}")
#     j+=1

In [7]:
# plt.figure(figsize=(16,28))
# path = "/kaggle/input/UBC-OCEAN/train_images"
# j=1
# for img, lb in zip(train_df_tma['image_id_path'],train_df_tma['label']):
#     plt.subplot(7,4,j)
#     path = os.path.join("/kaggle/input/UBC-OCEAN/train_images",img)
#     image = plt.imread(path)
#     image = plt.imshow(image)
#     plt.title(f"Label:{lb}")
#     j+=1

In [8]:
# # Create directories for resized training and validation images
# os.makedirs("/kaggle/working/resized_train", exist_ok=True)
# os.makedirs("/kaggle/working/resized_val", exist_ok=True)

# # Initialize sums and normalizer
# sums, sums_squared, normalizer = 0, 0, 0

# # Resize and save training images, and calculate mean and std
# for idx, row in train_df.head().iterrows():
#     img_path = get_train_file_path(row['image_id'])
#     img = cv2.imread(img_path)
    
#     # Convert to float and normalize
#     img = img.astype(np.float32) / 255.0
    
#     # Resize image to fit VGG16 input size
#     img_resized = cv2.resize(img, (224, 224))

#     # Update sums for mean and std calculation
#     sums += np.sum(img_resized)
#     sums_squared += np.sum(np.square(img_resized))
#     normalizer += img_resized.size  # Add the number of pixels in the image to the normalizer

#     save_path = f"/kaggle/working/resized_train/{row['image_id']}.png"
#     cv2.imwrite(save_path, img_resized * 255)  # Multiply by 255 to revert normalization

# # Calculate mean and std
# mean = sums / normalizer
# std = np.sqrt((sums_squared / normalizer) - np.square(mean))

# # Resize and save validation images
# for idx, row in val_df.head().iterrows():
#     img_path = get_train_file_path(row['image_id'])
#     img = cv2.imread(img_path)
    
#     # Convert to float and normalize
#     img = img.astype(np.float32) / 255.0
    
#     # Resize image to fit VGG16 input size
#     img_resized = cv2.resize(img, (224, 224))
    
#     save_path = f"/kaggle/working/resized_val/{row['image_id']}.png"
#     cv2.imwrite(save_path, img_resized * 255)  # Multiply by 255 to revert normalization


In [9]:
# Adding image path
df['image_id_path'] = np.where(df['is_tma'], 
                               df['image_id'].astype(str) + ".png", 
                               df['image_id'].astype(str) + "_thumbnail.png")


In [10]:
# Split dataset into training and validation
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)


In [11]:
# Differentiate between TMA and non-TMA
train_df_no_tma = train_df[train_df['is_tma']==False]
train_df_tma = train_df[train_df['is_tma']==True]

val_df_no_tma = val_df[val_df['is_tma']==False]
val_df_tma = val_df[val_df['is_tma']==True]

In [12]:
# Initialize sums for mean and std calculation
sums, sums_squared, normalizer = 0, 0, 0

# Paths
tumbnail_path = "/kaggle/input/UBC-OCEAN/train_thumbnails"
train_path = "/kaggle/input/UBC-OCEAN/train_images"

# Function to process each dataframe
def process_dataframe(df, folder_name, is_tma=False):
    global sums, sums_squared, normalizer  # Declare as global to modify
    labels_and_paths = []
    source_path = train_path if is_tma else tumbnail_path
    
    for idx, row in df.iterrows():
        img_path = os.path.join(source_path, row['image_id_path'])
        img = cv2.imread(img_path)
        
        # Convert to float and normalize
        img = img.astype(np.float32) / 255.0
        
        # Resize image to fit VGG16 input size
        img_resized = cv2.resize(img, (224, 224))
        
        if folder_name == 'train':
            # Update sums for mean and std calculation
            sums += np.sum(img_resized)
            sums_squared += np.sum(np.square(img_resized))
            normalizer += img_resized.size

        save_path = f"/kaggle/working/{folder_name}/{row['image_id_path']}"
        cv2.imwrite(save_path, img_resized * 255)
        
        labels_and_paths.append({'path': save_path, 'label': row['label']})
    
    return pd.DataFrame(labels_and_paths)


In [13]:
# Process and save train and validation images
train_no_tma_labels_df = process_dataframe(train_df_no_tma, 'train')
train_tma_labels_df = process_dataframe(train_df_tma, 'train', is_tma=True)
val_no_tma_labels_df = process_dataframe(val_df_no_tma, 'val')
val_tma_labels_df = process_dataframe(val_df_tma, 'val', is_tma=True)

In [14]:
# Combine the dataframes
train_labels_df = pd.concat([train_no_tma_labels_df, train_tma_labels_df])
val_labels_df = pd.concat([val_no_tma_labels_df, val_tma_labels_df])

In [15]:
# Calculate mean and std
mean = sums / normalizer
std = np.sqrt((sums_squared / normalizer) - np.square(mean))


In [16]:
# Save labels and paths
train_labels_df.to_csv('/kaggle/working/train_labels_and_paths.csv', index=False)
val_labels_df.to_csv('/kaggle/working/val_labels_and_paths.csv', index=False)

In [17]:
print(mean, std)

0.46951395000692525 0.40392743228535105
